**SA367 &#x25aa; Mathematical Models for Decision Making &#x25aa; Spring 2022 &#x25aa; Uhan**

# Lesson 9. The Self-Sufficient Marine 

## The problem

__Self-sufficiency__ is the ability to carry out a mission without external support or aid, which is critical for operations in environments with limited infrastructure and logistical support.
Such scenarios are common in expeditionary missions for the Marines.
Perhaps the most fundamental question of self-sufficiency for these missions is whether a squad can be successful with the items that the Marines are able to carry.

Suppose you have a list of the items that a Marine may carry for a particular mission. 
In addition, suppose you have for each item:

- the __weight__ of the item, in units of 0.1 lbs

- the __value__ of carrying one unit of the item for the mission, on a scale from 0 to 10

- __lower and upper bounds__ on how many of the item needs to be carried

- whether the item is __partially shared__: that is, if the item can be shared among a few Marines if needed (e.g. binoculars) 

- whether the item is __shared__: that is, if an item should be used by an entire squad (e.g. metal detector)

Assume that a Marine can carry 65 lbs, at most 2 *types* of partially shared items, and 0 shared items.
The objective is to determine how many of each item a Marine should carry in order to maximize the total value of the items.

Formulate this problem as a dynamic program by giving its shortest/longest path representation.

*Note.* This problem is a simpler variant of the one studied in this paper:

> J. Simon, A. Apte, E. Reginier. An application of the multiple knapsack problem: the self-sufficient marine. *European Journal of Operational Research* 256: 868-876 (2017).

## Setting up the data

- In the same folder as this notebook, there is a CSV file named `hot-SOP.csv`, that contains the data described above
    - This data is a modified version of the data used in the paper by Simon, Apte, and Reginier (2017) cited above
    - The items are based on the Standard Operating Procedures policy for hot climate training for hiking 20 kilometers
    - The value assigned to each item was determined by analyzing the preferences of many individual Marines


- Let's take a look using Pandas


- First, let's import Pandas:

In [ ]:
import pandas as pd

- Now we can read the CSV file into a Pandas DataFrame:

In [ ]:
df = pd.read_csv('hot-SOP.csv')

- Let's peek at the first 5 rows of the DataFrame:

- We see that whether an item is shared or not is indicated by a 1 or 0
    - Same thing goes for whether an item is partially shared or not


- We'll use the __index__ to refer to the different items:
    - e.g., item 2 is the BAT and HIIDE System


- Recall that we can convert DataFrame columns to dictionaries using `.to_dict()` 

- Let's create some dictionaries that will help us access the different pieces of information about each item: 

In [ ]:
item = df['Item'].to_dict()
weight = df['Weight'].to_dict()
value = df['Value'].to_dict()
lower = df['Lower'].to_dict()
upper = df['Upper'].to_dict()
shared = df['Shared'].to_dict()
partial_shared = df['PartialShared'].to_dict()

- If this worked correctly, we should be able to get the weight and value of the BAT and HIIDE System as follows:

## Solving the DP

- There are three important constants in our problem: the number of items, the weight capacity, and the number of partially shared items allowed 

- Let's create variables to hold these constants

- This way, we can easily adapt our code to accommodate similar DPs

- Next, let's import `networkx` and `bellmanford`:

In [ ]:
import networkx as nx
import bellmanford as bf

- As usual, we start with an empty digraph:

- Next, let's add the stage-state nodes:

- We can't forget the "end" node!

- Quick check: how many nodes do we have in our graph?

- Now it's time to add the edges

- Let's start with the edges corresponding to the decision of how many of each item to carry:

In [ ]:
# Add edges corresponding to how many of item t to carry
            
            # If item t is shared, don't consider it
            
            # Otherwise...
            
                # Let x be the number of item t to carry
                # x must be between the lower and upper bounds for item t
                    
                    # Weight capacity becomes... 
                    
                    # Partially shared item capacity becomes...
                    # Recall that we only care about the number of partially shared item *types* 
                    
                    # Add edge only if there's enough capacity
                    
                    # Don't forget we're maximizing the total value of the items carried!

- Now we can add the edges from the last stage to the end node

- Quick check: how many edges do we have in our graph?

- Finally, let's solve the shortest path problem we've constructed using the Bellman-Ford algorithm:

In [ ]:
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, N1, N2), target='end', weight='length')

print(f'Negative cycle? {negative_cycle}')
print(f'Shortest path length: {length}')
print(f'Shortest path: {nodes}')

- It's easy to see what the maximum total value of the items carried is... 

- However, how many of each item should we carry to get this maximum total value?

- Instead of reading through the path of 50-ish nodes to figure out how many of each item to carry, let's write some code to do this for us

- The $t$th node in the shortest path corresponds to stage/item $t$

- We can compute how many of item $t$ to carry as follows:

$$
\frac{(n_1 \mbox{ from stage } t) - (n_1 \mbox{ from stage } t + 1)}{\mbox{weight of item }  t} 
$$

- So...

In [ ]:
# Go through each stage in the shortest path:
    
    # Node from stage t
    
    # Node from stage t + 1
    
    # Compute number of item t to carry
    
    # Print information

## Incorporating shared items

- We ignored the possibility of carrying shared items in our model so far


- Suppose we can carry at most 2 types of partially shared items and at most 1 type of shared item


- How can we modify our dynamic program to accommodate this? Write a new dynamic program on paper

- Once you've figured out how you need to change our dynamic program, how can you modify the code above to solve the new dynamic program? 

---

## Problems

### Problem 1 (Solving the airlift planning problem)

Solve the dynamic program we formulated in Example 1 of Lesson 8 using Python. What is the maximum possible total capability value? Which requirements should be put on the plane to achieve this maximum total capability value?

_Write your notes here. Double-click to edit._